https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT
#from biobert and trained on all MIMIC notes (MIMIC database contains extensive information on patient medications and diagnoses)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------- User prompt ----------------------------- #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# --------------------------- Generation params -------------------------- #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------- Load model & tokenizer ----------------------- #
MODEL_ID = "emilyalsentzer/Bio_ClinicalBERT"          # causal LM; swap for any that fits your GPU
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE.type == "cuda" else torch.float32,
).to(DEVICE).eval()

# ----------------------------- Tokenization ----------------------------- #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
input_ids      = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ------------------------------ Inference ------------------------------- #
with torch.no_grad():
    output_ids = model.generate(
        input_ids           = input_ids,
        attention_mask      = attention_mask,
        max_new_tokens      = MAX_NEW_TOKENS,
        do_sample           = True,
        temperature         = TEMPERATURE,
        top_p               = TOP_P,
        pad_token_id        = tokenizer.eos_token_id
    )

# ---------------------------- Post-processing --------------------------- #
generated_text = tokenizer.decode(
    output_ids[0][input_ids.shape[-1]:],  # slice off the prompt
    skip_special_tokens=True
).strip()

print("GENERATED TEXT:\n", generated_text)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

GENERATED TEXT:
 ................................................................................................................................................................................................................................................................


https://huggingface.co/emilyalsentzer/Bio_Discharge_Summary_BERT
#similar but trained on only discharge summaries from MIMIC notes, summary includes hospital course, but i think bioClinical is better than this

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------- User prompt ----------------------------- #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# --------------------------- Generation params -------------------------- #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------- Load model & tokenizer ----------------------- #
MODEL_ID = "emilyalsentzer/Bio_Discharge_Summary_BERT"          # causal LM; swap for any that fits your GPU
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE.type == "cuda" else torch.float32,
).to(DEVICE).eval()

# ----------------------------- Tokenization ----------------------------- #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
input_ids      = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ------------------------------ Inference ------------------------------- #
with torch.no_grad():
    output_ids = model.generate(
        input_ids           = input_ids,
        attention_mask      = attention_mask,
        max_new_tokens      = MAX_NEW_TOKENS,
        do_sample           = True,
        temperature         = TEMPERATURE,
        top_p               = TOP_P,
        pad_token_id        = tokenizer.eos_token_id
    )

# ---------------------------- Post-processing --------------------------- #
generated_text = tokenizer.decode(
    output_ids[0][input_ids.shape[-1]:],  # slice off the prompt
    skip_special_tokens=True
).strip()

print("GENERATED TEXT:\n", generated_text)


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

GENERATED TEXT:
 ................................................................................................................................................................................................................................................................


https://huggingface.co/google/medgemma-4b-it
https://ollama.com/alibayram/medgemma
#medgemma

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import os
# ----------------------------- User prompt ----------------------------- #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# --------------------------- Generation params -------------------------- #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------- Load model & tokenizer ----------------------- #

MODEL_ID = "google/medgemma-4b-it"          # causal LM; swap for any that fits your GPU
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE.type == "cuda" else torch.float32,
).to(DEVICE).eval()

# ----------------------------- Tokenization ----------------------------- #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
input_ids      = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ------------------------------ Inference ------------------------------- #
with torch.no_grad():
    output_ids = model.generate(
        input_ids           = input_ids,
        attention_mask      = attention_mask,
        max_new_tokens      = MAX_NEW_TOKENS,
        do_sample           = True,
        temperature         = TEMPERATURE,
        top_p               = TOP_P,
        pad_token_id        = tokenizer.eos_token_id
    )

# ---------------------------- Post-processing --------------------------- #
generated_text = tokenizer.decode(
    output_ids[0][input_ids.shape[-1]:],  # slice off the prompt
    skip_special_tokens=True
).strip()

print("GENERATED TEXT:\n", generated_text)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

https://huggingface.co/nicoboss/Qwen-3-32B-Medical-Reasoning
#finetuned qwen model, seems like it can help

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------- User prompt ----------------------------- #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# --------------------------- Generation params -------------------------- #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------- Load model & tokenizer ----------------------- #
MODEL_ID = "nicoboss/Qwen-3-32B-Medical-Reasoning"          # causal LM; swap for any that fits your GPU
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE.type == "cuda" else torch.float32,
).to(DEVICE).eval()

# ----------------------------- Tokenization ----------------------------- #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
input_ids      = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ------------------------------ Inference ------------------------------- #
with torch.no_grad():
    output_ids = model.generate(
        input_ids           = input_ids,
        attention_mask      = attention_mask,
        max_new_tokens      = MAX_NEW_TOKENS,
        do_sample           = True,
        temperature         = TEMPERATURE,
        top_p               = TOP_P,
        pad_token_id        = tokenizer.eos_token_id
    )

# ---------------------------- Post-processing --------------------------- #
generated_text = tokenizer.decode(
    output_ids[0][input_ids.shape[-1]:],  # slice off the prompt
    skip_special_tokens=True
).strip()

print("GENERATED TEXT:\n", generated_text)

https://huggingface.co/epfl-llm/meditron-7b

#llama2-7b finetuned on medical corpus, including PubMed abstracts and other med documents

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------- User prompt ----------------------------- #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# --------------------------- Generation params -------------------------- #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------- Load model & tokenizer ----------------------- #
MODEL_ID = "epfl-llm/meditron-7b"          # causal LM; swap for any that fits your GPU
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE.type == "cuda" else torch.float32,
).to(DEVICE).eval()

# ----------------------------- Tokenization ----------------------------- #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
input_ids      = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ------------------------------ Inference ------------------------------- #
with torch.no_grad():
    output_ids = model.generate(
        input_ids           = input_ids,
        attention_mask      = attention_mask,
        max_new_tokens      = MAX_NEW_TOKENS,
        do_sample           = True,
        temperature         = TEMPERATURE,
        top_p               = TOP_P,
        pad_token_id        = tokenizer.eos_token_id
    )

# ---------------------------- Post-processing --------------------------- #
generated_text = tokenizer.decode(
    output_ids[0][input_ids.shape[-1]:],  # slice off the prompt
    skip_special_tokens=True
).strip()

print("GENERATED TEXT:\n", generated_text)

https://huggingface.co/docs/transformers/en/model_doc/biogpt
#GPT 2 on biod=medical tasks and docs

**Absolute Mess**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------- User prompt ----------------------------- #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# --------------------------- Generation params -------------------------- #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------- Load model & tokenizer ----------------------- #
MODEL_ID = "microsoft/biogpt"          # causal LM; swap for any that fits your GPU
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE.type == "cuda" else torch.float32,
).to(DEVICE).eval()

# ----------------------------- Tokenization ----------------------------- #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
input_ids      = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ------------------------------ Inference ------------------------------- #
with torch.no_grad():
    output_ids = model.generate(
        input_ids           = input_ids,
        attention_mask      = attention_mask,
        max_new_tokens      = MAX_NEW_TOKENS,
        do_sample           = True,
        temperature         = TEMPERATURE,
        top_p               = TOP_P,
        pad_token_id        = tokenizer.eos_token_id
    )

# ---------------------------- Post-processing --------------------------- #
generated_text = tokenizer.decode(
    output_ids[0][input_ids.shape[-1]:],  # slice off the prompt
    skip_special_tokens=True
).strip()

print("GENERATED TEXT:\n", generated_text)

https://huggingface.co/BioMistral/BioMistral-7B
#Mistral trained on PubMed data

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------- User prompt ----------------------------- #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# --------------------------- Generation params -------------------------- #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------- Load model & tokenizer ----------------------- #
MODEL_ID = "BioMistral/BioMistral-7B"          # causal LM; swap for any that fits your GPU
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE.type == "cuda" else torch.float32,
).to(DEVICE).eval()

# ----------------------------- Tokenization ----------------------------- #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
input_ids      = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ------------------------------ Inference ------------------------------- #
with torch.no_grad():
    output_ids = model.generate(
        input_ids           = input_ids,
        attention_mask      = attention_mask,
        max_new_tokens      = MAX_NEW_TOKENS,
        do_sample           = True,
        temperature         = TEMPERATURE,
        top_p               = TOP_P,
        pad_token_id        = tokenizer.eos_token_id
    )

# ---------------------------- Post-processing --------------------------- #
generated_text = tokenizer.decode(
    output_ids[0][input_ids.shape[-1]:],  # slice off the prompt
    skip_special_tokens=True
).strip()

print("GENERATED TEXT:\n", generated_text)

https://huggingface.co/clinicalnlplab/me-llama
#llama2 but on clinical notes and biomedical data

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------- User prompt ----------------------------- #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# --------------------------- Generation params -------------------------- #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------- Load model & tokenizer ----------------------- #
MODEL_ID = "clinicalnlplab/me-llama"          # causal LM; swap for any that fits your GPU
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE.type == "cuda" else torch.float32,
).to(DEVICE).eval()

# ----------------------------- Tokenization ----------------------------- #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
input_ids      = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ------------------------------ Inference ------------------------------- #
with torch.no_grad():
    output_ids = model.generate(
        input_ids           = input_ids,
        attention_mask      = attention_mask,
        max_new_tokens      = MAX_NEW_TOKENS,
        do_sample           = True,
        temperature         = TEMPERATURE,
        top_p               = TOP_P,
        pad_token_id        = tokenizer.eos_token_id
    )

# ---------------------------- Post-processing --------------------------- #
generated_text = tokenizer.decode(
    output_ids[0][input_ids.shape[-1]:],  # slice off the prompt
    skip_special_tokens=True
).strip()

print("GENERATED TEXT:\n", generated_text)

#other models which are computationally a bit expensive also dont know much on their capabilities on medicinal data, but are medical llms
https://huggingface.co/mradermacher/JSL-Med-Mistral-24B-V1-Slerp-i1-GGUF
https://huggingface.co/mradermacher/JSL-MedQwen-14b-reasoning-i1-GGUF
https://huggingface.co/mradermacher/JSL-Med-Phi-3.5-Mini-v3-i1-GGUF
https://huggingface.co/mradermacher/Llama-3.1-8B-UltraMedical-i1-GGUF

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------- User prompt ----------------------------- #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# --------------------------- Generation params -------------------------- #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------- Load model & tokenizer ----------------------- #
MODEL_ID = "mradermacher/JSL-MedQwen-14b-reasoning-i1-GGUF"          # causal LM; swap for any that fits your GPU
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE.type == "cuda" else torch.float32,
).to(DEVICE).eval()

# ----------------------------- Tokenization ----------------------------- #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
input_ids      = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ------------------------------ Inference ------------------------------- #
with torch.no_grad():
    output_ids = model.generate(
        input_ids           = input_ids,
        attention_mask      = attention_mask,
        max_new_tokens      = MAX_NEW_TOKENS,
        do_sample           = True,
        temperature         = TEMPERATURE,
        top_p               = TOP_P,
        pad_token_id        = tokenizer.eos_token_id
    )

# ---------------------------- Post-processing --------------------------- #
generated_text = tokenizer.decode(
    output_ids[0][input_ids.shape[-1]:],  # slice off the prompt
    skip_special_tokens=True
).strip()

print("GENERATED TEXT:\n", generated_text)

OSError: There was a specific connection error when trying to load mradermacher/JSL-MedQwen-14b-reasoning-i1-GGUF:
401 Client Error: Unauthorized for url: https://huggingface.co/mradermacher/JSL-MedQwen-14b-reasoning-i1-GGUF/resolve/main/config.json (Request ID: Root=1-68bebc2c-5872689e7ca03ed433e7191c;60a2f6a4-3a9e-428d-8dfa-c9df410ae1d8)

Invalid credentials in Authorization header

#also check this out
https://huggingface.co/blog/leaderboard-medicalllm

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------- User prompt ----------------------------- #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# --------------------------- Generation params -------------------------- #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------- Load model & tokenizer ----------------------- #
MODEL_ID = "microsoft/BioGPT-Large"          # causal LM; swap for any that fits your GPU
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE.type == "cuda" else torch.float32,
).to(DEVICE).eval()

# ----------------------------- Tokenization ----------------------------- #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
input_ids      = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ------------------------------ Inference ------------------------------- #
with torch.no_grad():
    output_ids = model.generate(
        input_ids           = input_ids,
        attention_mask      = attention_mask,
        max_new_tokens      = MAX_NEW_TOKENS,
        do_sample           = True,
        temperature         = TEMPERATURE,
        top_p               = TOP_P,
        pad_token_id        = tokenizer.eos_token_id
    )

# ---------------------------- Post-processing --------------------------- #
generated_text = tokenizer.decode(
    output_ids[0][input_ids.shape[-1]:],  # slice off the prompt
    skip_special_tokens=True
).strip()

print("GENERATED TEXT:\n", generated_text)

#Biobert models, finetuned models of these below,
https://huggingface.co/dmis-lab/biobert-large-cased-v1.1-squad

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------- User prompt ----------------------------- #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# --------------------------- Generation params -------------------------- #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------- Load model & tokenizer ----------------------- #
MODEL_ID = "dmis-lab/biobert-large-cased-v1.1-squad"          # causal LM; swap for any that fits your GPU
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE.type == "cuda" else torch.float32,
).to(DEVICE).eval()

# ----------------------------- Tokenization ----------------------------- #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
input_ids      = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ------------------------------ Inference ------------------------------- #
with torch.no_grad():
    output_ids = model.generate(
        input_ids           = input_ids,
        attention_mask      = attention_mask,
        max_new_tokens      = MAX_NEW_TOKENS,
        do_sample           = True,
        temperature         = TEMPERATURE,
        top_p               = TOP_P,
        pad_token_id        = tokenizer.eos_token_id
    )

# ---------------------------- Post-processing --------------------------- #
generated_text = tokenizer.decode(
    output_ids[0][input_ids.shape[-1]:],  # slice off the prompt
    skip_special_tokens=True
).strip()

print("GENERATED TEXT:\n", generated_text)

OSError: There was a specific connection error when trying to load dmis-lab/biobert-large-cased-v1.1-squad:
401 Client Error: Unauthorized for url: https://huggingface.co/dmis-lab/biobert-large-cased-v1.1-squad/resolve/main/config.json (Request ID: Root=1-68bebbeb-475c8976041b953007830969;28b2b3d9-76b4-40cb-9e48-0f13a10a50cc)

Invalid credentials in Authorization header